<h3 style="text-align: center;"><span style="color:red">This example that can be overwritten by container updates.</span></h3>
<h4 style="text-align: center;"><span style="color:red">Please create a new notebook if you plan to make changes.</span></h4>

## Intro

This example jupyter file demonstrates basic [stable diffusion](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/stable_diffusion_2#texttoimage) functionality.

## Getting started

Run each cell to step through the process, including (first-time only) downloading a model to use.
Note: Run cells, in order, at least once, to set up the environment.  Once set up, you can change the prompt and rerun just that cell.  If you make changes in a cell, that cell should be rerun for the changes to take effect.

In [ ]:
from dataclasses import dataclass, field, asdict
import torch

@dataclass
class Model:
    name: str
    repo_id: str
    variant: str = None
    torch_dtype: str = None
    foldername: str = field(init=False)

    def __post_init__(self):
        # Set the value of foldername based on repo_id
        self.foldername = self.repo_id.split("/")[1]

    def fp16(self):
        self.variant = "fp16"
        self.torch_dtype = torch.float16
        return self

    def pretrain_options(self):
        return self._filter(["name", "repo_id", "foldername"])

    def _filter(self, excludes):
        # Return the members that aren't set to None or in excludes
        return {k:v for k,v in asdict(self).items() if (v is not None and k not in excludes)}

models = {
    "sd2"        : Model("sd2", "stabilityai/stable-diffusion-2-base"),
    "sd-turbo"   : Model("sd-turbo", "stabilityai/sd-turbo"),
    "sdxl"       : Model("sdxl", "stabilityai/stable-diffusion-xl-base-1.0"),
}

model = models["sd2"].fp16()
model

In [ ]:
from diffusers import AutoPipelineForText2Image

pipe = AutoPipelineForText2Image.from_pretrained(model.repo_id, **model.pretrain_options())
pipe.to("cuda")

prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."
image = pipe(prompt=prompt).images[0]
image